# Using GridSearchCV with Tensorflor: Regression

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

from scikeras.wrappers import KerasRegressor

In [10]:
# df = pd.read_excel("housing_data.xlsx")
url = "https://raw.githubusercontent.com/ie-ai-class/ai-2566-2/main/T6%20-%20ANN/S02%20-%20Regression/housing_data.csv"
df = pd.read_csv(url)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [11]:
# Shuffle data
df = df.sample(frac=1, random_state=1)

# Extract data

colsX = [i for i in df.columns if i != "MEDV"]
X = df[colsX].values
y = df["MEDV"].values
print(X.shape)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Scale y data
nm = MinMaxScaler()
y_train = y_train.reshape(-1, 1)  # Required for sklearn api
y_test = y_test.reshape(-1, 1)
nm.fit(y_train)
y_train_scaled = nm.transform(y_train)
y_test_scaled = nm.transform(y_test)

(506, 13)


In [12]:
def get_clf_model(hidden_layer_sizes, meta, compile_kwargs):

    model = tf.keras.Sequential()
    input_layer = tf.keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(input_layer)

    for i, hidden_layer_size in enumerate(hidden_layer_sizes):
        hidden_layer = tf.keras.layers.Dense(
            units=hidden_layer_size, activation="relu", name=f"hidden{i+1}"
        )
        model.add(hidden_layer)

    output_layer = tf.keras.layers.Dense(units=1, name="output")
    model.add(output_layer)

    model.compile(
        optimizer=compile_kwargs["optimizer"],
        loss="mean_squared_error",
        metrics=[tf.keras.metrics.mean_absolute_percentage_error],
    )

    return model

In [13]:
tf.keras.backend.clear_session()
tf.keras.backend.set_epsilon(1)

In [14]:
# Note that I am setting validation_split=0, because already use GridSearchCV which has cross-validation.

clf = KerasRegressor(
    model=get_clf_model,
    hidden_layer_sizes=(16,),
    optimizer="adam",
    optimizer__learning_rate=0.001,
    epochs=200,
    batch_size=16,
    validation_split=0.2,  
    verbose=1,
)

In [15]:
set1 = {
    "optimizer__learning_rate": [0.01, 0.1],
    "hidden_layer_sizes": [(16, 3), (8, 3)],
}

param_grid = [set1]

In [16]:
gs = GridSearchCV(estimator=clf, 
                  param_grid=param_grid, 
                  scoring='neg_mean_squared_error', 
                  cv=5,
                  n_jobs=-1)

In [17]:
gs.fit(X_train_std,y_train)

Epoch 1/200
18/18 [==============================] - 1s 12ms/step - loss: 599.9071 - mean_absolute_percentage_error: 97.2642 - val_loss: 468.7636 - val_mean_absolute_percentage_error: 90.0378
Epoch 2/200
18/18 [==============================] - 0s 3ms/step - loss: 529.8844 - mean_absolute_percentage_error: 86.7896 - val_loss: 373.8861 - val_mean_absolute_percentage_error: 77.2157
Epoch 3/200
18/18 [==============================] - 0s 3ms/step - loss: 371.4660 - mean_absolute_percentage_error: 68.5073 - val_loss: 183.8917 - val_mean_absolute_percentage_error: 53.1798
Epoch 4/200
18/18 [==============================] - 0s 3ms/step - loss: 141.0454 - mean_absolute_percentage_error: 40.6955 - val_loss: 72.7354 - val_mean_absolute_percentage_error: 38.9139
Epoch 5/200
18/18 [==============================] - 0s 3ms/step - loss: 61.6746 - mean_absolute_percentage_error: 28.6118 - val_loss: 44.4555 - val_mean_absolute_percentage_error: 32.2183
Epoch 6/200
18/18 [============================

GridSearchCV(cv=5,
             estimator=KerasRegressor(batch_size=16, epochs=200, hidden_layer_sizes=(16,), model=<function get_clf_model at 0x00000176A9B753A0>, optimizer='adam', optimizer__learning_rate=0.001, validation_split=0.2),
             n_jobs=-1,
             param_grid=[{'hidden_layer_sizes': [(16, 3), (8, 3)],
                          'optimizer__learning_rate': [0.01, 0.1]}],
             scoring='neg_mean_squared_error')

In [18]:
df = pd.DataFrame(gs.cv_results_)
df = df.sort_values(by=['rank_test_score'])
display(df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,20.149172,0.225232,0.206127,0.050131,"(8, 3)",0.01,"{'hidden_layer_sizes': (8, 3), 'optimizer__lea...",-18.916374,-19.816294,-17.995046,-13.184765,-16.526369,-17.287770,2.322086,1
1,20.113577,0.253949,0.225431,0.042137,"(16, 3)",0.1,"{'hidden_layer_sizes': (16, 3), 'optimizer__le...",-11.590407,-31.522482,-26.945024,-20.004303,-23.142973,-22.641038,6.733871,2
3,11.585274,4.247267,0.103682,0.053680,"(8, 3)",0.1,"{'hidden_layer_sizes': (8, 3), 'optimizer__lea...",-21.931763,-25.406710,-38.883499,-15.388269,-19.386060,-24.199260,8.037746,3
0,20.179420,0.207322,0.235651,0.039468,"(16, 3)",0.01,"{'hidden_layer_sizes': (16, 3), 'optimizer__le...",-19.712135,-20.701989,-95.798145,-103.226060,-16.081027,-51.103871,39.624780,4


In [19]:
print(gs.best_score_)
print(gs.best_params_)

-17.287769574065674
{'hidden_layer_sizes': (8, 3), 'optimizer__learning_rate': 0.01}


In [20]:
y_pred = gs.predict(X_test_std)
print(mean_absolute_percentage_error(y_test, y_pred))

10/10 [==============================] - 0s 666us/step
0.12311050304987613
